<a href="https://colab.research.google.com/github/xBosse/notebooks/blob/main/scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import

In [ ]:
! pip install datasets transformers seqeval
import transformers, datasets

import json, random
!git clone https://github.com/xBosse/datasets.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 33.5 MB/s 
     |████████████████████████████████| 4.2 MB 50.1 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 212 kB 61.9 MB/s 
     |████████████████████████████████| 140 kB 58.6 MB/s 
     |████████████████████████████████| 86 kB 7.5 MB/s 
     |████████████████████████████████| 1.1 MB 44.6 MB/s 
     |████████████████████████████████| 596 kB 47.4 MB/s 
     |████████████████████████████████| 127 kB 50.0 MB/s 
     |████████████████████████████████| 6.6 MB 15.1 MB/s 
     |████████████████████████████████| 94 kB 1.1 MB/s 
     |████████████████████████████████| 144 kB 53.8 MB/s 
     |████████████████████████████████| 271 kB 89.0 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=06dcde6297

#Pipeline / Modell initialisieren

Hier wird das zu evaluierende Modell geladen und über eine Pipeline verwendet.

model_name ist gleich der Name des zu evaluierende Modell auf Huggingface.

Modell v1.0: tbosse/bert-base-german-cased-finetuned-subj_v6_7Epoch
Modell v1.1: tbosse/bert-base-german-cased-finetuned-subj_v6_7Epoch_v2
Modell v1.2: tbosse/bert-base-german-cased-finetuned-subj_v6_7Epoch_v3

Modell v2.0: tbosse/bert-base-german-cased-noisy-pretrain-fine-tuned 
Modell v2.1: tbosse/bert-base-german-cased-noisy-pretrain-fine-tuned_v1.1
Modell v2.2: tbosse/bert-base-german-cased-noisy-pretrain-fine-tuned_v1.2



In [ ]:
from transformers import AutoTokenizer,AutoModelForTokenClassification

model_name = "tbosse/bert-base-german-cased-finetuned-subj_v6_7Epoch"

#model_name = "tbosse/bert-base-german-cased-noisy-pretrain-fine-tuned"


model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)

from transformers import pipeline

classifier = pipeline("token-classification", model=model, tokenizer=tokenizer) # , padding=True, truncation=True, max_length=512
#result_test = classifier("Ich mag keine Birnen.")

Downloading:   0%|          | 0.00/799 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/414M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/327 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/249k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/709k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

# Trainingsdatensatz laden

In [ ]:

from datasets import load_dataset, load_metric, ClassLabel, Value, Features

import datasets

#DATENSATZ

#test_url = "https://raw.githubusercontent.com/xBosse/datasets/main/test_shata14.txt"

test_url = "https://raw.githubusercontent.com/xBosse/datasets/main/test_steps16.txt"

# eigener Code

classFeatures = datasets.Features({
    'words': datasets.Sequence(Value("string")),
    'subj': datasets.Sequence(ClassLabel(names=['O', 'B-SUBJ', 'I-SUBJ']))
})

# eigener Parameter

datasets = load_dataset('json', data_files = {"test": test_url}, features=classFeatures)


Using custom data configuration default-31bffc2a6b3e4cab


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-31bffc2a6b3e4cab/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

# mit Modell Testdatensatz klassifizieren

In [ ]:
#joining words from Dataset into sentences in list
test_strings = datasets["test"]["words"] #########
test_str = []
for line in test_strings:
  test_str.append(" ".join(line))

#classify all sentences
results_classifier = []
for test in test_str:
  results_classifier.append(classifier(test))





# Dataset-Formatieren:  Token-splitting mit Filler-removal

Die durch den Tokenizer des Modells getrennten Subwörter werden zu den Originalwörtern zusammengefügt

In [ ]:
#################################  Stellen an denen Tokens zusammmen gefügt werden müssen: 


################################# 
# gesplittete Wörter zusammensetzen und Labels kombinieren
pre_alligned_results_classifier = []

for x in range(len(results_classifier)):
  line = []
  for y in range(len(results_classifier[x])):
    
    word = results_classifier[x][y]["word"]
    label = results_classifier[x][y]["entity"]
    if "0" in label:
      label = "0"
    elif "1" in label:
      label = "1"
    elif "2" in label:
      label = "2"
    else: 
      label = "0"

    if "##" in word: #Wenn TOKEN = SUBWORD

      #subword dem letzten Wort anhängen
      subword = word[2:]
      line[len(line)-1][0] = line[len(line)-1][0] + subword
      
      #label 
      line[len(line)-1][1] = line[len(line)-1][1] + label
 
    else: # Wenn TOKEN = eigenes Wort

      # neues Token dict erstellen und line anhängen
      token = []
      
      token.append(word)
      token.append(label)

      line.append(token)

  pre_alligned_results_classifier.append(line)






In [ ]:
# test-set mit Platzhalter Tokens auffüllen

####### Labels des Test Datentsatzes
test_words = datasets["test"]["words"]
test_labels = datasets["test"]["subj"]

alligned_test_words = []

for x in range(len(pre_alligned_results_classifier)):

  #print("x: " + str(x) )
  line = []
  i = 0
  for y in range(len(pre_alligned_results_classifier[x])):

    #print("y: " + str(y) + " - i: " + str(i))
    res_word = pre_alligned_results_classifier[x][y][0]

    if len(test_words[x]) > (y - i):

      test_word = test_words[x][y - i]
      test_label = test_labels[x][y - i]
    else:
      test_word = ""
      test_label = "3"
    

    if res_word in test_word:
      # neues Wort:
      token = []
      token.append(test_word)
      token.append(test_label)
      line.append(token)
    
    else:
      # platzhalter Token:
      token = []
      token.append("FILLER")
      token.append("3")
      line.append(token)
      i += 1

    
  alligned_test_words.append(line)

In [ ]:
# platzhalter Entfernen, damit Resultset angleichen

alligned_results_classifier = []

for x in range(len(alligned_test_words)):
  #print("x: " + str(x) )
  line = []
  for y in range(len(alligned_test_words[x])):
    #print("y: " + str(y) )
    res_word = pre_alligned_results_classifier[x][y][0]
    res_label = pre_alligned_results_classifier[x][y][1]

    test_word = alligned_test_words[x][y][0]

    if test_word == "FILLER":
      #print("filler")
      # letzem wort anhängen
      line[len(line)-1][0] = line[len(line)-1][0] + res_word        
      #label 
      line[len(line)-1][1] = line[len(line)-1][1] + res_label

    else:
      #print("new Word")
      # neues Wort:
      token = []
      token.append(res_word)
      token.append(res_label)
      line.append(token)


  alligned_results_classifier.append(line)



# Labels der zusammengefügten Subwörter entscheiden

In [ ]:
###################       # Majority vote:

majority_vote_results = []

for line in alligned_results_classifier:
  new_line = []
  for word in line:
    token = []
    label = word[1]
    if len(word[1]) > 1:
      frequency = [0,0,0]
      for char in word[1]:
        if char == "0":
          frequency[0] += 1
        elif char == "1":
          frequency[1] += 1
        elif char == "2":
          frequency[2] += 1
      label = str(frequency.index(max(frequency)))

    token.append(word[0])
    token.append(label)
    new_line.append(token)
  majority_vote_results.append(new_line)    

      

In [ ]:

###################       # Feste Rangordnung 2 > 1 > 0

rangordnung_results = []

for line in alligned_results_classifier:
  new_line = []
  for word in line:
    token = []
    label = word[1]
    if len(word[1]) > 1:
      frequency = [0,0,0]
      for char in word[1]:
        if char == "0":
          frequency[0] += 1
        elif char == "1":
          frequency[1] += 1
        elif char == "2":
          frequency[2] += 1

      if frequency[2] > 0:
        label = "2"     
      elif frequency[1] > 0:
        label = "1"
      else:
        label = "0"

    token.append(word[0])
    token.append(label)
    new_line.append(token)
  rangordnung_results.append(new_line)  


In [ ]:
###################       # First Subword only

first_subword_results = []

for line in alligned_results_classifier:
  new_line = []
  for word in line:
    token = []
    label = word[1]
    label = label[0]
    token.append(word[0])
    token.append(label)
    new_line.append(token)
  first_subword_results.append(new_line)     

 

#Multi Token Subj Expr

Um zu Unterscheiden welche Tokens zu welchem subjektiven Ausdruck gehören wird eine Liste aus den subjektiven Ausdrücken und deren Positionen gebildet. 

Beispielshaft für das Problem:

Label-Liste: 0,1,0,1,0,2

Label 1 steht für den Anfang eines subjektiven Ausdruckes, Label 2 bedeutet es gehört zu einem subjektiven Ausdruck.

Zu welcher 1 gehört die 2? 
zum Bewerten ist das wichtig, da falls eine 1 und die 2 falsch klassifiziert werden, und diese zu einem subjektiven Ausdruck gehören, dies ein False Negative wäre. Falls diese zu seperaten subjektiven Ausdrücken gehören sind das zwei False Negatives.

In [ ]:
# Reading as Txt file
with open('datasets/steps16_v1.jsonl') as f:
    lines = f.readlines()
# Closing file
f.close()



# Jsonl to Json
import json

string = "{ \"Steps16\": ["
for x in lines:
  string += x + ","

string = string[:-1]
string += "]}"

data = json.loads(string)


# Json to Lists / Getting only  SubjExpr-Indexes
scorer_set = []

i = -1
last_words = data["Steps16"][0]["seq_words"]
scorer_line = []

for line in data["Steps16"]:
  if line["seq_words"] == last_words:
    scorer_line.append(line["pred_sense"][0])
  else:
    last_words = line["seq_words"]
    i += 1
    scorer_set.append(scorer_line)
    scorer_line = []
    scorer_line.append(line["pred_sense"][0])
scorer_set.append(scorer_line)


########### Duplikate aus scorer_set löschen??? zB in Zeile 0: [[3], [9], [3], [7]] ##############


# Evaluieren

In [ ]:
#Get int Arrays of labels from self-classified results

def reformatResults(resultSet):
  reformattedResults = []

  for line in resultSet:
    refLine = []
    for token in line:
      refLine.append(int(token[1]))
      
    reformattedResults.append(refLine)
  
  return reformattedResults


# Methode anwenden
reformatted_rangordnung_results = reformatResults(rangordnung_results)
reformatted_majority_vote_results = reformatResults(majority_vote_results)
reformatted_first_subword_results = reformatResults(first_subword_results)

####### Labels des Test Datentsatzes
test_labels = datasets["test"]["subj"]
test_words = datasets["test"]["words"]

In [ ]:
################                      SCORING MIT MULTI-TOKEN SUBJEXPR                      ################  

#Accuracy TP+TN/all

#Precision TP/TP+FP = TP/all predictet Positives

#Recall TP/TP+FN = TP/all actual Positives

#F1 2*Prec*Rec / (Prec+Rec)

def scorerMulti(scorerSet, testSet, results):
  tp = 0
  fp = 0
  fn = 0

  
  for x in range(len(results)):
    
    s = scorerSet[x]
    r = results[x]
    y2 = 0
    # Alle SubjExpr des ScorerSets/GoldStandarts durchgehen
    for subj in s:
      i = 1
      matches = 0
      for x in range(len(subj)):
        res_label = r[subj[x]-1]
        if res_label == i:    
          matches += 1
        i = 2
        
      if matches == len(subj): # TruePositiv = jeder vollständig erkannte subjektive Ausdruck
        tp += 1


      if matches < len(subj): # FalseNegativ = jeder subjektive Ausdruck der nicht vollständig erkannt wurde
        fn += 1
        

      y2 += 1


  for x in range(len(results)):
    for y in range(len(results[x])):
      
      t = testSet[x][y] # testset label
      r = results[x][y] # result label

      if t!=r and (r==1 or r==2) :    
        fp += 1                       #FalsePositiv = jedes Label 0, welches falsch klassifiziert wurde
        

  p = tp / (tp + fp)
  r = tp / (tp + fn)
  f = 2*p*r / (p+r)


  scores = {
      "precision": p,
      "recall": r,
      "f1-score": f
  }

  print("tp: " + str(tp))
  print("fp: " + str(fp))
  print("fn: " + str(fn))
  print("--------------")

  return scores

In [ ]:
scoresM_first = scorerMulti(scorer_set, test_labels, reformatted_first_subword_results)
print(scoresM_first)

tp: 1794
fp: 389
fn: 623
--------------
{'precision': 0.8218048557031608, 'recall': 0.7422424493173355, 'f1-score': 0.7799999999999998}


In [ ]:
scoresM_rang = scorerMulti(scorer_set, test_labels, reformatted_rangordnung_results)
print(scoresM_rang)

tp: 1834
fp: 410
fn: 583
--------------
{'precision': 0.8172905525846702, 'recall': 0.7587918907736864, 'f1-score': 0.7869555889294142}


In [ ]:
scoresM_vote = scorerMulti(scorer_set, test_labels, reformatted_majority_vote_results)
print(scoresM_vote)

tp: 1780
fp: 357
fn: 637
--------------
{'precision': 0.8329433785680861, 'recall': 0.7364501448076127, 'f1-score': 0.7817303469477382}


# Vergleich der Ergebnisse

In [ ]:
test_labels = datasets["test"]["subj"]
test_words = datasets["test"]["words"]
result_labels = reformatted_first_subword_results


In [ ]:
type(result_labels[0][0])

int

In [ ]:


for x in range(len(test_words)):
  if str(test_labels[x]) != str(result_labels[x]):
    print(test_words[x])
    print("GoldStd: " + str(test_labels[x]))
    print("Results: " + str(result_labels[x]) + "\n")

['Zuerst', 'zu', 'den', 'ökologischen', 'Gründen', '-', 'ich', 'möchte', 'im', 'Zusammenhang', 'mit', 'der', 'Klimapolitik', 'einige', 'Aspekte', 'in', 'Erinnerung', 'rufen', '-', ':', 'In', 'der', 'Wissenschaft', 'besteht', 'ein', 'grosser', 'Konsens', 'über', 'die', 'wesentlichen', 'Zusammenhänge', 'der', 'Klimaerwärmung', '.']
GoldStd: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0]
Results: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

['Auch', 'wachsen', 'die', 'Erkenntnisse', ',', 'wonach', 'eine', 'Erwärmung', 'um', 'mehr', 'als', '2', 'Grad', 'für', 'die', 'Gesellschaft', 'gravierende', 'Auswirkungen', 'hätte', '.']
GoldStd: [0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Results: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0]

['Das', 'Problem', 'ist', ',', 'dass', 'es', 'für', 'die', 'Gesellschaft', 'umso', 'schwieriger', 'und', 'teurer',

# Test Zone: Token unterschiede ausgeben

In [ ]:
len(test_labels)

578

In [ ]:
i = 36

In [ ]:
datasets["test"]["words"][i]

In [ ]:
majority_vote_results[i]

Zeichen, an denen der Tokenizer nicht splitten sollte:

 "-" zwischen Buchstaben, nach Buchstaben

"," zwischen Zahlen

"&quot;Der" "&quot;"  	'&quot;Man', '&quot;' 		'&quot;Schweigen', '&quot;'	'&quot;Livre', 'neutralité&quot;' 	

".." 

"20."  Punkt nach Zahl: Daten usw

"Fr." Punkt nach Buchstaben: Abkürzungen



In [ ]:
i=0
unique_tokens = []
for x in range(len(test_labels)): 
  
  #print("Index: " + str(x) + ", Test: " + str(len(test_labels[x])) + ", Results: " + str(len(reformatted_majority_vote_results[x])) + ", Diff: " + str(len(test_labels[x]) - len(reformatted_majority_vote_results[x])) )
  if len(test_labels[x]) > len(reformatted_majority_vote_results[x]):
    i += 1
    #print(datasets["test"]["words"][x])
    u_tokens = datasets["test"]["words"][x]
    prnt = []
    for token in majority_vote_results[x]:
      prnt.append(token[0])
      if token[0] in u_tokens:
        u_tokens.remove(token[0])

    #print(prnt)
    print("Diff: " + str(len(test_labels[x]) - len(reformatted_majority_vote_results[x])) )
    print("Index " + str(x) + ": " + str(u_tokens))
    unique_tokens.append(u_tokens)
    print("")
print(i)
print(unique_tokens)

0
[]


In [ ]:
i=0
unique_tokens = []
for x in range(len(test_labels)): 
  
  #print("Index: " + str(x) + ", Test: " + str(len(test_labels[x])) + ", Results: " + str(len(reformatted_majority_vote_results[x])) + ", Diff: " + str(len(test_labels[x]) - len(reformatted_majority_vote_results[x])) )
  if len(test_labels[x]) < len(reformatted_majority_vote_results[x]):
    i += 1
    print(datasets["test"]["words"][x])
    u_tokens = datasets["test"]["words"][x]
    prnt = []
    for token in majority_vote_results[x]:
      prnt.append(token[0])
      if token[0] in u_tokens:
        u_tokens.remove(token[0])

    print(prnt)
    print("Diff: " + str(len(test_labels[x]) - len(reformatted_majority_vote_results[x])) )
    print("Unique Tokens in sentence " + str(x) + ": " + str(u_tokens))
    unique_tokens.append(u_tokens)
    print("")
print(i)
print(unique_tokens)

0
[]


In [ ]:
#ONLY PRINT CASE: 

i=0
unique_tokens = []
for x in range(len(test_labels)): 
  
  #print("Index: " + str(x) + ", Test: " + str(len(test_labels[x])) + ", Results: " + str(len(reformatted_majority_vote_results[x])) + ", Diff: " + str(len(test_labels[x]) - len(reformatted_majority_vote_results[x])) )
  if len(test_labels[x]) < len(reformatted_majority_vote_results[x]):
    #i += 1
    #print(datasets["test"]["words"][x])
    u_tokens = datasets["test"]["words"][x]
    prnt = []
    for token in majority_vote_results[x]:
      prnt.append(token[0])
      if token[0] in u_tokens:
        u_tokens.remove(token[0])
  

    for tok in u_tokens:
      if("-" in tok):
        i += 1
        print("Token: " + str(tok))
        print(datasets["test"]["words"][x])
        print(prnt)       
        print("Diff: " + str(len(test_labels[x]) - len(reformatted_majority_vote_results[x])) )
        print("Index: " + str(x) )
        print("")
        unique_tokens.append(u_tokens)
        

print("Anzahl: " + str(i))
print(unique_tokens)


Anzahl: 0
[]


# Alter Code

Resultset formatieren: gesplittete Tokens zusammensetzen

In [ ]:
###############################################################

     ### ALTER ANSATZ: AUSNAHMEFÄLLE DES TOKENIZERS EINZELND REPARIEREN ######

###############################################################


    ####### &qout;

    elif last_word is "&" and word is "qout":
      # letzem wort anhängen
      line[len(line)-1][0] = line[len(line)-1][0] + word        
      #label 
      line[len(line)-1][1] = line[len(line)-1][1] + label

    elif last_word is "qout" and word is ";":
      # letzem wort anhängen
      line[len(line)-1][0] = line[len(line)-1][0] + word        
      #label 
      line[len(line)-1][1] = line[len(line)-1][1] + label

    elif last_word is ";" and (word is "minimal" or word is "fremdnützige"):
      # letzem wort anhängen
      line[len(line)-1][0] = line[len(line)-1][0] + word        
      #label 
      line[len(line)-1][1] = line[len(line)-1][1] + label

    elif last_word is "Forschung" and word is "&":
      # letzem wort anhängen
      line[len(line)-1][0] = line[len(line)-1][0] + word        
      #label 
      line[len(line)-1][1] = line[len(line)-1][1] + label




    ###### , Komma
    
    if "," in word:
      # letzem wort anhängen
      line[len(line)-1][0] = line[len(line)-1][0] + word        
      #label 
      line[len(line)-1][1] = line[len(line)-1][1] + label
      continue

    if "," in last_word and word.isdigit():
      # letzem wort anhängen
      line[len(line)-1][0] = line[len(line)-1][0] + word        
      #label 
      line[len(line)-1][1] = line[len(line)-1][1] + label
      continue



    ######### .

    elif last_word is "." and word is ".":
      # letzem wort anhängen
      line[len(line)-1][0] = line[len(line)-1][0] + word        
      #label 
      line[len(line)-1][1] = line[len(line)-1][1] + label
     
    elif last_word is "." and (word is "In" or word is "Dazu"):      
      # letzem wort anhängen
      line[len(line)-1][0] = line[len(line)-1][0] + word        
      #label 
      line[len(line)-1][1] = line[len(line)-1][1] + label

    elif (last_word.isdigit() or last_word is "bzw" or last_word is "usw" or last_word is "B") and word is ".":
      # letzem wort anhängen
      line[len(line)-1][0] = line[len(line)-1][0] + word        
      #label 
      line[len(line)-1][1] = line[len(line)-1][1] + label



# gesplittete Sub-Wörter zusammensetzen und Labels kombinieren 
pre_alligned_results_classifier = []
alligned_results_classifier = []

for x in range(len(results_classifier)):
  line = []
  for y in range(len(results_classifier[x])):
    
    word = results_classifier[x][y]["word"]
    label = results_classifier[x][y]["entity"]
    if "0" in label:
      label = "0"
    elif "1" in label:
      label = "1"
    elif "2" in label:
      label = "2"
    else: 
      label = "0"

    if "##" in word: #Wenn TOKEN = SUBWORD

      #subword dem letzten Wort anhängen
      subword = word[2:]
      line[len(line)-1][0] = line[len(line)-1][0] + subword
      
      #label 
      line[len(line)-1][1] = line[len(line)-1][1] + label
 
    else: # Wenn TOKEN = eigenes Wort

      # neues Token dict erstellen und line anhängen
      token = []
      
      token.append(word)
      token.append(label)

      line.append(token)

  pre_alligned_results_classifier.append(line)

#################################################################

# Wörter die an Sonderzeichen gesplittet wurden zusammenfügen
for x in range(len(pre_alligned_results_classifier)):
  line = []
  for y in range(len(pre_alligned_results_classifier[x])):
    
    word = pre_alligned_results_classifier[x][y][0]
    label = pre_alligned_results_classifier[x][y][1]

    last_word = ""
    if(y > 0): #if current word not first
      last_word = pre_alligned_results_classifier[x][y-1][0]

    next_word = ""
    if((y+1) < len(pre_alligned_results_classifier[x])):   #if current word not last
      next_word = pre_alligned_results_classifier[x][y+1][0]





    ######### /


    if "/" in word:
      # letzem wort anhängen
      line[len(line)-1][0] = line[len(line)-1][0] + word        
      #label 
      line[len(line)-1][1] = line[len(line)-1][1] + label
      continue
    

    if  "Ems" in word:
      # letzem wort anhängen
      line[len(line)-1][0] = line[len(line)-1][0] + word        
      #label 
      line[len(line)-1][1] = line[len(line)-1][1] + label
      continue

    ####### z.B.
    if  (x == 73 or x == 537) and (last_word == "z" and word == "."):
       # letzem wort anhängen
      line[len(line)-1][0] = line[len(line)-1][0] + word        
      #label 
      line[len(line)-1][1] = line[len(line)-1][1] + label
      continue

    if  (x == 73 or x == 537) and last_word == "." and word == "B":
       # letzem wort anhängen
      line[len(line)-1][0] = line[len(line)-1][0] + word        
      #label 
      line[len(line)-1][1] = line[len(line)-1][1] + label
      continue

    if  (x == 73 or x == 537 or x == 68 or x == 396) and last_word == "B" and word == ".":
       # letzem wort anhängen
      line[len(line)-1][0] = line[len(line)-1][0] + word        
      #label 
      line[len(line)-1][1] = line[len(line)-1][1] + label
      continue



    ####### - Bindestrich

    prehyphen = ["Uno", "2000", "Watt" , "50", "Know", "How", "New", "Deal", "SP", "Wirtschafts", "WAK", "FDP", "zivil", "Haftpflicht", "Regierungs", "Vor", "Gotthard", "Alpenschutz", "x", "EU", "Knack", "Skyguide", "Neat", "Staat", "Tausch", "Verkaufs", "Online", "Hans", "Umwelt", "Swissair", "BDP", "Kauf", "Wald", "Besitz", "Landwirtschafts", "Nazi", "UBS", "Fluglärm", "Lärm"]
    posthyphen = ["Klimabericht", "Watt", "Gesellschaft", "jährigen", "how", "Deal", "Politik", "Fraktion", "NR", "Liberale", "Basistunnels", "Initiative", "mal", "Kommission", "Gebäude", "Mehrkosten", "Privatperson", "Handel", "Jörg", "Groundings", "Zeit", "Debakel", "Protestorganisationen"]

    if ("-" in last_word) and (word in posthyphen):     
      # letzem wort anhängen
      line[len(line)-1][0] = line[len(line)-1][0] + word        
      #label 
      line[len(line)-1][1] = line[len(line)-1][1] + label
      continue

    if ("-" in word) and (last_word in prehyphen) and not (x==270 and last_word == "EU") and not (x==328 and last_word == "Wald"):    
      # letzem wort anhängen
      line[len(line)-1][0] = line[len(line)-1][0] + word        
      #label 
      line[len(line)-1][1] = line[len(line)-1][1] + label
      continue
 
 


    # Wenn TOKEN = eigenes Wort

    # neues Token dict erstellen und line anhängen
    token = []
      
    token.append(word)
    token.append(label)

    line.append(token)

  alligned_results_classifier.append(line)


In [ ]:
################                                            ################  
################          GANZE LOGIK NOCHMAL ÜBERDENKEN    ################  
################                                            ################  


#Accuracy TP+TN/all

#Precision TP/TP+FP = TP/all predictet Positives

#Recall TP/TP+FN = TP/all actual Positives

#F1 2*Prec*Rec / (Prec+Rec)



def scorer(testSet, results):
  tp = 0
  fp = 0
  tn = 0
  fn = 0

  i = 0


  #TP
  case11 = 0
  case22 = 0

  #TN
  case00 = 0
  
  #FP
  case12 = 0
  case21 = 0

  case01 = 0
  case02 = 0

  #FN
  case10 = 0
  case20 = 0


  for x in range(len(results)):
    for y in range(len(results[x])):

      #  Vergleich: testSet=results
      # Nur relevant, falls 1 oder 2 enthalten ist
      # TP: 1=1,2=2, vielleicht auch 1=2, 2=1
      # FP: 0=1, 0=2
      # TN: 0=0??? vielleicht nicht zählen?
      # FN: 1=0, 2=0


      t = testSet[x][y] # testset label
      r = results[x][y] # result label
      if t==r:
        
        if t==0:
          tn += 1


          case00 += 1
        else:
          tp += 1
          if t==1:
            case11 += 1
          else:
            case22 += 1
      
      else:
        if t==0:
          fp += 1

          if r==1:
            case01 += 1
          else:
            case02 += 1

        elif 0==r:
          fn += 1

          if t==1:
            case10 += 1
          else:
            case20 += 1
        
        else:
          # case: 1=2, 2=1 
          ############ zählt zu TP oder FP???: ############
          fp += 1

          i += 1
          if t == 1:
            case12 += 1
          else:
            case21 += 1            

  all = tp + fp + fn + tn ### TN: 0=0  weggelassen??
      
  a =  (tp + tn) / all ### TN weggelassen
  p = tp / (tp + fp)
  r = tp / (tp + fn)
  f = 2*p*r / (p+r)


  scores = {
      "accuracy": a,
      "precision": p,
      "recall": r,
      "f1-score": f
  }

  print("tp: " + str(tp))
  print("fp: " + str(fp))
  print("tn: " + str(tn))
  print("fn: " + str(fn))
  print("--------------")

  #print("2-1+1-2: " + str(i))
  print("0-0: " + str(case00))
  print("1-1: " + str(case11))
  print("2-2: " + str(case22))

  print("0-1: " + str(case01))
  print("0-2: " + str(case02))

  print("1-0: " + str(case10))
  print("2-0: " + str(case20))

  print("1-2: " + str(case12))
  print("2-1: " + str(case21))


  print("--------------")

  return scores

In [ ]:
scores_majority_vote = scorer(test_labels, reformatted_majority_vote_results)
print(scores_majority_vote)

scores_rangordnung = scorer(test_labels, reformatted_rangordnung_results)
print(scores_rangordnung)

scores_first = scorer(test_labels, reformatted_first_subword_results)
print(scores_first)